## Installing fastText

The first step of this tutorial is to install and build fastText.

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
!cd fastText-0.9.2 && make && pip install .

--2021-10-06 11:54:28--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [suivant]
--2021-10-06 11:54:29--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Résolution de codeload.github.com (codeload.github.com)… 140.82.121.10
Connexion à codeload.github.com (codeload.github.com)|140.82.121.10|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : non indiqué [application/zip]
Enregistre : «v0.9.2.zip.1»

v0.9.2.zip.1            [    <=>             ]   4,17M  6,90MB/s    ds 0,6s    

2021-10-06 11:54:29 (6,90 MB/s) - «v0.9.2.zip.1» enregistré [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
replace fastText-0.9.2/.circleci/cmake_test.sh? [y]es, [n]o, 

In [5]:
import fasttext

### Download directly with command line the english dataset

In [6]:
!cd fastText-0.9.2 && python3 download_model.py en

File exists. Use --overwrite to download anyway.


### Import HuggingFace

In [7]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Loading the dataset "IMDB"

In [8]:
from datasets import load_dataset

Using the split argument, we can split the imdb into two separate dataset.

In [9]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/clara/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/clara/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


*train_dataset* is a class with two attributes :
- Features which contains two features : text and label (our x_train and our y_train)
- The number of rows in our dataset

In [10]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


If we take the first sample of our training dataset, we can see the first review and the label (positive or negative) according to that review.

In [11]:
print(train_dataset[0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


### Preprocessing the dataset

With the review we previously saw, the review are definitely not preprocessed. There is still some html tags etc.. 
We definitely to work on those.

In [12]:
!pip install contractions
!pip install beautifulsoup4
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [120]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk.stem import PorterStemmer

import tqdm
from tqdm import tqdm

In [121]:
def strip_html_tags(text: str) -> str:
    '''
    Strip html tags from the input text.

            Parameters:
                    text (str): A text.

            Returns:
                    stripped_text (str): Text completely stripped without html tags.
    '''
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [122]:
def remove_accented_chars(text: str) -> str:
    '''
    Remove accented characters from the text.

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text which does not contains accented characters.
    '''
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [123]:
def expand_contractions(text: str) -> str:
    '''
    Expand contractions like "you're" to "you are".

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text with no contractions.
    '''
    return contractions.fix(text)

In [124]:
def expand_contractions(text: str) -> str:
    '''
    Expand contractions like "you're" to "you are".

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text with no contractions.
    '''
    return contractions.fix(text)

In [125]:
def remove_special_characters(text: str , remove_digits: bool) -> str:
    '''
    Remove special characters from the text.

            Parameters:
                    text (str): A text.
                    remove_digits (bool): True if you want to remove digits, else False.

            Returns:
                    text (str): Text which does not contains special characters (digits, etc...).
    '''
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

In [126]:
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()

def lemming(text: str, lemming: bool) -> str:
    '''
    The words in our text input will be lemmatized to their root form. 
    Furthermore, the stop words and the words with the length less than 4 will be removed from the corpus.

            Parameters:
                    text (str): A text.
                    lemming (bool): True if you want the lemmitization, if not False.

            Returns:
                    text (str): Lemmed text.
    '''
    
    if (lemming):
        tokens = text.split()
        tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

[nltk_data] Downloading package wordnet to /home/clara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [127]:
ps = PorterStemmer()

def stemming(text: str, stemming: bool) -> str:
    '''
    Stemming is the process of producing morphological variants of a root/base word.
    We will use this function to do the same for our document.

            Parameters:
                    text (str): A text.
                    lemming (bool): True if you want the stemmatization, if not False.

            Returns:
                    text (str): Stemmed text.
    '''
    if (stemming):
        tokens = text.split()
        tokens = [ps.stem(word) for word in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

In [128]:
en_stop = set(nltk.corpus.stopwords.words('english'))

def stop_words(text: str, stop_words: bool) -> str:
    '''
    ????

            Parameters:
                    text (str): A text.
                    stop_words (bool): True if you want the delete them, if not False.

            Returns:
                    text (str): Text without stop words.
    '''
    if (stop_words):
        tokens = text.split()
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

In [129]:
def pre_process_document(document: str, lemm: bool, stemm: bool, sw: bool) -> str:
    '''
    Preprocess the document to remove all the unnecessary information.

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text pre-processed.
    '''
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    document = lemming(document, lemm)
    document = stemming(document, stemm)
    
    document = stop_words(document, sw)

    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [54]:
y_train = train_dataset['label']

In [30]:
y_test = test_dataset['label']

In [135]:
def create_imdb_train_file(train_dataset, y_train: list,
                           lemm: bool = False, stemm:bool = False, stop_words:bool = False):
    '''
    Create the imdb train file according to the fasttext documentation.
    
             Parameters:
                     train_dataset: Training dataset
                     y_train : Label for the training dataset
    '''
    
    if (lemm):
        f = open("imdb_train_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_train_stemmed.txt", "w")
        
    elif (stop_words):
        f = open("imdb_train_sw.txt", "w")        

    else:
        f = open("imdb_train.txt", "w") 
    
    for i in tqdm(range(train_dataset.num_rows)):
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
    
    f.close()

In [136]:
def create_imdb_test_file(test_dataset, y_test: list, lemm: bool = False, stemm: bool = False, stop_words: bool = False):
    '''
    Create the imdb test file according to the fasttext documentation.
    
             Parameters:
                     test_dataset: Training dataset
                     y_test: Label for the testing dataset
    '''
    if (lemm):
        f = open("imdb_test_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_test_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_test_sw.txt", "w")        

    else:
        f = open("imdb_test.txt", "w") 
    
    for i in tqdm(range(test_dataset.num_rows)):
        if y_test[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(test_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_test[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(test_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
            
    f.close()

In [137]:
def create_imdb_train_and_validation_file(train_dataset, y_train: list, lemm: bool = False,
                                          stemm: bool = False, stop_words: bool = False):
    '''
    Create the imdb validation file according to the fasttext documentation.
    
             Parameters:
                     train_dataset: Training dataset
                     y_train: Label for the training dataset
    '''
    i = 0
    
    if (lemm):
        f = open("imdb_validation_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_validation_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_validation_sw.txt", "w")        

    else:
        f = open("imdb_validation.txt", "w") 
    
    for i in tqdm(range(5000)):
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
    
    f.close()
    
    if (lemm):
        f = open("imdb_train_splited_with_the_validation_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_train_splited_with_the_validation_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_train_splited_with_the_validation_sw.txt", "w")        

    else:
        f = open("imdb_train_splited_with_the_validation.txt", "w") 
    
    while i != train_dataset.num_rows:
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        i += 1
    
    f.close()

In [97]:
create_imdb_train_file(train_dataset, y_train)
create_imdb_train_and_validation_file(train_dataset, y_train)
create_imdb_test_file(test_dataset, y_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:18<00:00, 1352.43it/s]


## Let's train the model using fasttext.

##### 1) Lemming

In [98]:
create_imdb_train_file(train_dataset, y_train, lemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, lemm = True)
create_imdb_test_file(test_dataset, y_test, lemm = True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:44<00:00, 564.23it/s]


In [99]:
%%time
model = fasttext.train_supervised(input="imdb_train_lemmed.txt")

Read 5M words
Number of words:  93399
Number of labels: 2


CPU times: user 12.5 s, sys: 249 ms, total: 12.7 s
Wall time: 5.33 s


Progress: 100.0% words/sec/thread: 2221161 lr:  0.000000 avg.loss:  0.404254 ETA:   0h 0m 0s


In [100]:
res = model.test("imdb_test.txt")
print(res)

(25000, 0.84256, 0.84256)


##### 2) Stemming

In [101]:
create_imdb_train_file(train_dataset, y_train, stemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stemm = True)
create_imdb_test_file(test_dataset, y_test, stemm = True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [01:47<00:00, 232.38it/s]


In [102]:
%%time
model = fasttext.train_supervised(input="imdb_train_stemmed.txt")

Read 5M words
Number of words:  74057
Number of labels: 2
Progress:  98.4% words/sec/thread: 2469537 lr:  0.001571 avg.loss:  0.264267 ETA:   0h 0m 0s

CPU times: user 11.6 s, sys: 224 ms, total: 11.8 s
Wall time: 4.61 s


Progress: 100.0% words/sec/thread: 2446215 lr:  0.000000 avg.loss:  0.263788 ETA:   0h 0m 0s


In [103]:
res = model.test("imdb_test.txt")
print(res)

(25000, 0.81796, 0.81796)


##### 3) Stop Words

In [105]:
create_imdb_train_file(train_dataset, y_train, stop_words = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stop_words = True)
create_imdb_test_file(test_dataset, y_test, stop_words = True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:19<00:00, 1289.47it/s]


In [106]:
%%time
model = fasttext.train_supervised(input="imdb_train_sw.txt")

Read 2M words
Number of words:  99724
Number of labels: 2
Progress:  94.0% words/sec/thread: 1949998 lr:  0.006038 avg.loss:  0.248016 ETA:   0h 0m 0s

CPU times: user 6.98 s, sys: 163 ms, total: 7.14 s
Wall time: 2.93 s


Progress: 100.0% words/sec/thread: 1902247 lr:  0.000000 avg.loss:  0.242394 ETA:   0h 0m 0s


In [107]:
res = model.test("imdb_test.txt")
print(res)

(25000, 0.87176, 0.87176)


##### 4) None

In [108]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt")

Read 5M words
Number of words:  102284
Number of labels: 2
Progress:  95.2% words/sec/thread: 2450738 lr:  0.004787 avg.loss:  0.298446 ETA:   0h 0m 0s 0.015025 avg.loss:  0.318316 ETA:   0h 0m 0s

CPU times: user 11.9 s, sys: 179 ms, total: 12.1 s
Wall time: 4.73 s


Progress: 100.0% words/sec/thread: 2444660 lr:  0.000000 avg.loss:  0.294353 ETA:   0h 0m 0s


In [109]:
res = model.test("imdb_test.txt")
print(res)

(25000, 0.84588, 0.84588)


### Making the model better

#### How about N-grams ?

In [110]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", wordNgrams=2) 

Read 5M words
Number of words:  102284
Number of labels: 2
Progress:  98.3% words/sec/thread:  952230 lr:  0.001678 avg.loss:  0.403023 ETA:   0h 0m 0s

CPU times: user 30.8 s, sys: 516 ms, total: 31.3 s
Wall time: 11.7 s


Progress: 100.0% words/sec/thread:  949692 lr:  0.000000 avg.loss:  0.400764 ETA:   0h 0m 0s


In [111]:
model.test("imdb_test.txt")

(25000, 0.86824, 0.86824)

#### With more epochs and a smaller learning_rate ?

In [112]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.5, epoch=25, wordNgrams=2)

Read 5M words
Number of words:  102284
Number of labels: 2
Progress: 100.0% words/sec/thread:  884986 lr:  0.000275 avg.loss:  0.054940 ETA:   0h 0m 0s 14.0% words/sec/thread:  937863 lr:  0.430021 avg.loss:  0.283242 ETA:   0h 0m44sm43s

CPU times: user 2min 32s, sys: 2 s, total: 2min 34s
Wall time: 56.8 s


Progress: 100.0% words/sec/thread:  883876 lr:  0.000000 avg.loss:  0.054908 ETA:   0h 0m 0s


In [113]:
model.test("imdb_test.txt")

(25000, 0.89544, 0.89544)

#### With more epochs and a smaller smaller learning_rate ?

In [114]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.25, epoch=25, wordNgrams=2)

Read 5M words
Number of words:  102284
Number of labels: 2
Progress: 100.0% words/sec/thread:  925738 lr:  0.000071 avg.loss:  0.054127 ETA:   0h 0m 0s 30.7% words/sec/thread:  903140 lr:  0.173331 avg.loss:  0.152286 ETA:   0h 0m37s 0.113013 avg.loss:  0.093962 ETA:   0h 0m24s words/sec/thread:  915850 lr:  0.085151 avg.loss:  0.079338 ETA:   0h 0m18sm12s

CPU times: user 2min 29s, sys: 1.94 s, total: 2min 31s
Wall time: 54.4 s


Progress: 100.0% words/sec/thread:  924255 lr:  0.000000 avg.loss:  0.054114 ETA:   0h 0m 0s


In [115]:
model.test("imdb_test.txt")

(25000, 0.89024, 0.89024)

#### With more epochs and a bigger learning_rate ?

In [116]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=1.0, epoch=25, wordNgrams=2)

Read 5M words
Number of words:  102284
Number of labels: 2
Progress:  99.8% words/sec/thread:  922500 lr:  0.001503 avg.loss:  0.042932 ETA:   0h 0m 0s 23.9% words/sec/thread:  906174 lr:  0.760812 avg.loss:  0.163926 ETA:   0h 0m41s 72.2% words/sec/thread:  914289 lr:  0.277827 avg.loss:  0.058940 ETA:   0h 0m14s

CPU times: user 2min 29s, sys: 1.85 s, total: 2min 31s
Wall time: 54.5 s


Progress: 100.0% words/sec/thread:  922149 lr:  0.000000 avg.loss:  0.042871 ETA:   0h 0m 0s


In [117]:
model.test("imdb_test.txt")

(25000, 0.8914, 0.8914)

#### With the autotuneValidationfile (testing and performance purpose)

In [118]:
%%time
model = fasttext.train_supervised(input="imdb_train_splited_with_the_validation.txt", autotuneValidationFile='imdb_validation.txt')

Progress: 100.0% Trials:    9 Best score:  0.921800 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  90597
Number of labels: 2
Progress:  95.4% words/sec/thread: 1806095 lr:  0.008937 avg.loss:  0.388595 ETA:   0h 0m 0s

CPU times: user 13min 47s, sys: 7.62 s, total: 13min 54s
Wall time: 5min 7s


Progress: 100.0% words/sec/thread: 1784449 lr:  0.000000 avg.loss:  0.386236 ETA:   0h 0m 0s


In [119]:
model.test("imdb_test.txt")

(25000, 0.85736, 0.85736)

## Beating the baseline